In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import *

In [79]:
sub='401'
eyestring = '/Volumes/Voss_Lab/EEG/domcueMIXED_djb/eye.data/'
eyepath=Path(eyestring)

behavestring = '/Volumes/Voss_Lab/EEG/domcueMIXED_djb/behave.data/'
behavepath=Path(behavestring)
studyobj=[behavestring+sub+'study']
studypath=Path(studyobj[0])

colnames=['obj1','obj2','obj3','cuecond','loc1','x1','y1','loc2','x2','y2','loc3','x3','y3',
     'domRT','dom_resp','cond','block','tmp', 'tmp']
studyarray=pd.read_table(studypath,header=None ,names=colnames, index_col=None)
tmpmask=~studyarray.columns.str.contains('tmp')
studyarray=studyarray[studyarray.columns[tmpmask]]

trialnum = np.arange(1,len(studyarray)+1)
studyarray['trialnum'] = trialnum


/Users/drdj/.pyenv/versions/3.6.5/lib/python3.6/site-packages/pandas/io/parsers.py:709: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [ ]:
substrings=["*.asc"]
suball = []
subinfo = []
for s in substrings:
    for filepathobj in eyepath.glob(s):
        fname=filepathobj.name
        parts=fname.split(".")[0]
        subject=parts[:3]
        suball.append(subject)
        block=parts[3:4]
        subdict={"subject":subject, "block":block, "fname":fname}
        subinfo.append(subdict)
sublist = set(suball)
eye_sub = pd.DataFrame(subinfo)
eye_sub = eye_sub[eye_sub['subject']=='401']
eye_sub.sort_values(by=['fname'], inplace = True)
eye_sub.reset_index(inplace=True, drop=True)

In [49]:
def parse_eye_events_to_intline(line,extrainfo):
    efixspace=["","",""]
    eblinkspace=efixspace*2
    newline=line.split()
    if "EFIX" in line:
        newline.extend(efixspace)
    elif "EBLINK" in line:
        newline.extend(eblinkspace)
    newline.extend(extrainfo)
    return newline

def parse_eye_line(eye_sub, eyestring):
    """ parses each line of eye file for a given eye_phase_sub
    input one phase type list of files for a subs
    and the path to the file (in form of a string)
    outputs dataframe with all events in table
    """
    etypes=('ESACC','EFIX','EBLINK')
    study=[]
    restudy=[]
    blocks=eye_sub.block
    fnames=eye_sub.fname
    subjects=eye_sub.subject
    totalcount=0
    trialnum=0
    print(eyestring)
    for block,fname,subject in zip(blocks,fnames,subjects):
        path_file=eyestring+fname
        startcount=0
        p=Path(path_file)
        with p.open() as f:
            for line in f:
                if "START" in line:
                    totalcount=totalcount+1
                    startline=line.split()
                    starttime=int(startline[1])
                    if totalcount % 2:
                        trialnum = trialnum+1

                if any(e in line for e in etypes):
                    extrainfo=[starttime,trialnum,block,subject]
                    newline=parse_eye_events_to_intline(line,extrainfo)
                    if totalcount % 2:
                        restudy.append(newline)
                    else:
                        study.append(newline)
            print(trialnum, block, startcount)
    return study, restudy
study, restudy = parse_eye_line(eye_sub, eyestring)


def events_to_df(events):
    """ change raw events to data DataFrame
    then and change values to numeric"""

    eye_events_df=pd.DataFrame(events)
    eye_events_df=eye_events_df.apply(pd.to_numeric,errors='ignore')
    headers=["event","eye","start","end","duration",
    "xstart","ystart","xend","yend","?","?","trialstart",
    "trialnum","block","sub"]
    eye_events_df.columns=headers
    return eye_events_df

def eventsdf_cleanup(eye_events_df):
    """adjust trial start time, remove irrelevant values in fixation rows,
    and then delete excess columns"""

    eyedf_clean=eye_events_df.copy()

    eyedf_clean['start']=eyedf_clean['start']-eyedf_clean['trialstart']
    eyedf_clean['end']=eyedf_clean['end']-eyedf_clean['trialstart']

    efix_mask = (eyedf_clean["event"]=="EFIX")
    eyedf_clean.loc[efix_mask, 'xend'] = np.nan

    del eyedf_clean['trialstart']
    del eyedf_clean['?']
    del eyedf_clean['eye']

    return eyedf_clean

studydf = events_to_df(study)
studydf_clean = eventsdf_cleanup(studydf)

restudydf = events_to_df(restudy)
restudydf_clean = eventsdf_cleanup(restudydf)

/Volumes/Voss_Lab/EEG/domcueMIXED_djb/eye.data/
20 1 0
40 2 0
60 3 0
80 4 0
100 5 0
120 6 0
140 7 0
160 8 0


In [56]:
studydf_clean['phase']='study'
restudydf_clean['phase']='restudy'

In [ ]:
def set_behavior_path(sub, behavestring):
    extra='recogarray.txt'
    behaveobj=[behavestring+sub+extra]
    behavepath=Path(behaveobj[0])
    behavepath.exists()
    return behavepath

In [ ]:
def adjust_pres_coords(array,x,y,xmax=1920/2,ymax=1080/2):
    """adjustment for behavioral coords to match
    eye coords for presentation version of exp"""
    newarray=pd.DataFrame()
    newarray[x]=array[x]+xmax
    newarray[y]=(array[y]-ymax)*-1
    return newarray

def apply_adjust_pres_coords(behavearray):
    """applies adjust_pres_coords to all
    coords in behave array"""
    newloc1=adjust_pres_coords(behavearray,'x1','x1')
    newloc2=adjust_pres_coords(behavearray,'x2','x2')
    newloc3=adjust_pres_coords(behavearray,'x3','x3')
    newlocs=pd.concat([newloc1,newloc2,newloc3],axis=1)

    cols=newlocs.columns.tolist()
    for loc in cols:
        behavearray[loc]=newlocs[loc]
    return behavearray

In [ ]:
all = studydf_clean.append(restudydf_clean)
all.reset_index(inplace=True, drop=True)
